In [1]:
import numpy as np
from scipy.optimize import fsolve
import math

In [2]:
from scipy.special import psi

In [ ]:
#K - number of topics
K = 3
# N - number of character locations
N = 10
# V - number of vocabulary
V = 60

In [32]:
def compute_eta1(x):
    # x is the vector of \tilde(eta_1k)
    return np.mean(x)

In [83]:
def compute_eta2(x,y,z, K=3):
    # x is the vector of \tilde(eta_1k)
    # y is the vector of \tilde(eta_2k)
    # z is eta_1
    sum_ = 0
    for i in range(K):
        toadd = y[i]**2 + np.inner(x[k]-z, x[k]-z)
        sum_ += toadd
    return sum_/K

In [ ]:
def compute_nu1(x):
    # x is the vector of \tilde(nu_1n)
    return np.mean(x)

In [39]:
def compute_nu2(x,y,z, N=10):
    # x is the vector of \tilde(nu_1n)
    # y is the vector of \tilde(nu_2n)
    # z is nu_1
    sum_ = 0
    for i in range(K):
        toadd = y[i]**2 + np.inner(x[k]-z, x[k]-z)
        sum_ += toadd
    return sum_/N

In [ ]:
def compute_omega():
    return np.infty

In [67]:
def compute_phi(x0, V0= 60, K0 =3):
    # x is the matrix of \tilde(phi_kj)
    x = x0
    V=V0
    K=K0
    sol = optimize.root(lambda y: phi_MLE_func(x, y, V, K), x0 = [0.0])
    return sol.x

In [57]:
def phi_MLE_func(x, y, V = 60, K = 3):
    # x is the matrix of \tilde(phi_kj), KxV
    # y is phi
    # psi = digamma function
    sum_ = 0
    for k in range(K):
        temp1 = psi(sum(x[k,]))
        for v in range(V):
            toadd = psi(x[k,v]-temp1)
            sum_ += toadd
    return (psi(V*y)-psi(y))*K*V +sum_
    

In [147]:
def compute_teta1k(a0, b0, c0, d0, e0, f0, g0, N0, R0, K0, k0):
    # a is the vector of \tilde(eta_1k)
    # b is eta_1
    # c is eta_2
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # g is the matrix of \tilde(tau_nrk) NxRxK
    # N = 10
    # R - number of words in character i’s document
    a = a0
    b=b0
    c=c0
    d=d0
    e=e0
    f=f0
    g=g0
    N=N0
    R=R0
    K=K0
    k=k0
    sol = optimize.root(lambda x: teta1k_func(k,x, a, b, c, d, e, f, g, N, R, K), x0 = [0,0])
    return sol.x

In [154]:
g[1,1]

1

In [172]:
def teta1k_func(k,x, a, b, c, d, e, f, g, N, R, K):
    # k is the kth term of \tilde(eta_1k) we are asking for
    # x is \tilde(eta_1k) for this iteration
    # a is the vector of \tilde(eta_1k)
    # b is eta_1
    # c is eta_2
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # g is the matrix of \tilde(tau_nrk) NxRxK
    # N = 10
    # R - number of words in character i’s document
    # K = 3
    
    first_term = 2*(b-x)/(c**2)
    sum_ = [0,0]
    for n in range(N):
        
        temp1 = (math.dist(x, d[n]) + 2*(e[k]**2 + f[n]**2))**(-0.5)
        #print(temp1)
        temp2 = (x-d[n])/(math.dist(x, d[n]))
        #print(temp2)
        temp3 = math.exp(-1*((math.dist(x,d[n])+2*(e[k]**2+f[n]**2))**0.5))
        denominator = 0
        for c in range(K):
                temp4 = math.exp(-1*((math.dist(a[c],d[n])+2*(e[c]**2+f[n]**2))**0.5))
                denominator += temp4
        temp5 = 1- temp3/denominator
        print(temp5)
        for r in range(R[n]):
            toadd = g[n,r,k]*temp1*temp2*temp5
            #print(toadd)
            sum_ += toadd        
    return first_term - 0.5*sum_

In [80]:
def compute_teta2k(k0,, a0, c0, d0, e0,f0, g0, N0, R0, K0):
    k = k0
    a = a0
    c = c0
    d = d0
    e = e0
    f = f0
    g = g0
    N = N0
    R = R0
    K = K0
    sol = optimize.root(lambda x: teta1k_func(k,x, a, c, d, e, f, g, N, R, K), x0 = [0])    
    return sol.x

In [79]:
def teta2k_func(k, x, a, c, d, e,f, g, N, R, K):
    # k is the kth term of \tilde(eta_2k) we are asking for
    # x is \tilde(eta_2k)
    # a is the vector of \tilde(eta_1k)
    # c is eta_2
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # g is the matrix of \tilde(tau_nrk) NxRxk
    sum_ = 0
    first_term = 2/x
    second_term = 2*x/(c**2)
    
    for n in range(N):
        temp1 = ((math.dist(a[k], d[n]) + 2*(x**2 + f[n]**2))**(-0.5))
        denominator = 0
        temp3 = math.exp(-1*((math.dist(a[k],d[n])+2*(x**2+f[n]**2))**0.5))
        for c in range(K):
                temp4 = math.exp(-1*((math.dist(a[c],d[n])+2*(e[c]**2+f[n]**2))**(0.5)))
                denominator += temp4
        temp5 = 1- temp3/denominator
        for r in range(R[n]):
            toadd = g[n,r,k]*x*temp1*temp5
            sum_ += toadd
    
    return first_term -second_term -0.5*sum_
    

In [82]:
def compute_tnu1n(n0, a0, d0,e0, f0, g0, h0, w0, m0, y0,N0, R0, K0):
    n = n0
    a = a0
    d = d0
    e = e0
    f = f0
    g = g0
    h = h0
    w = w0
    m = m0
    y = y0
    N = N0
    R = R0
    K = K0
    sol = optimize.root(lambda x: tnu1n_func(n,x, a, d, e, f, g, h, w, m, y, N, R, K), x0 = [0,0])    
    return sol.x

In [81]:
def tnu1n_func(n,x, a, d, e, f, g, h, w, m, y, N, R, K):
    # n is the kth term of \tilde(nu_1n) we are asking for
    # x is \tilde(nu_1n)
    # a is the vector of \tilde(eta_1k)
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # g is the matrix of \tilde(tau_nrk) NxRxK
    # h is nu_1
    # m is nu_2
    # w is \tilde(omega)
    # y is connection matrix NxN
    first_term = 2*(h-x)/(m**2)
    denominator = 0
    for c in range(K):
            temp4 = math.exp(-1*((math.dist(a[c],x)+2*(e[c]**2+f[n]**2))**0.5))
            denominator += temp4
    result1 = 0       
    for r in range(R[n]):
        for k in range(K):
            f_term = g[n,r,k]*((math.dist(a[k], x) + 2*(e[k]**2 + f[n]**2))**0.5)* (x-a[k])/(math.dist(a[k], x))
            to_mult_1 = g[n,r,k]/denominator 
            to_mult_2 = 0
            for c in range(K):
                temp4 = math.exp(-1*((math.dist(a[c],x)+2*(e[c]**2+f[n]**2))**0.5))*((math.dist(a[c],x)+2*(e[c]**2+f[n]**2))**0.5)*(x-a[c])/(math.dist(x, a[c]))
                to_mult_2 += temp4
            s_term = to_mult_1*to_mult_2
            result1 += f_term-s_term
    result1 = result1/2
    result2 = 0
    for i in range(N):
        if i != n:
            to_mult_1 = ((math.dist(x, d[i]) + 2*(f[i]**2 + f[n]**2))**(0.5))*(x-d[i])/(math.dist(x, d[i]))
            to_mult_2 = y[n, i]*w - math.exp(w*(1-((math.dist(x, d[i]) + 2*(f[i]**2 + f[n]**2))**(0.5))))/(1+math.exp(w*(1-((math.dist(x, d[i]) + 2*(f[i]**2 + f[n]**2))**(0.5)))))
            result2 += to_mult_1*to_mult_2
    result2 = result2/2
    return first_term-result1 - result2

In [189]:
def compute_tnu2n(n, m, a,d, e,f,g, w, K, R, N):
    sol = optimize.root(lambda x: tnu2n_func(n, x, m, a,d, e,f,g, w, K, R, N), x0 = [0])    

In [188]:
def tnu2n_func(n, x, m, a,d, e,f,g, w, K, R, N):
    # x is \tilde(nu_2i) of interest
    # m is nu_2
    # g is the array of \tilde(tau_nrk) NxRxK
    # a is the array of \tilde(eta_1k)
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # w is \tilde(omega)
    first_term = 2/x - 2*x/(m**2)
    second_term= 0
    temp1 = 0
    temp3 = 0
    for c in range(K):
        temp2 = math.exp(-1* (math.dist(a[c], d[n]) + 2*(e[c]**2 + f[n]**2))**(0.5)) * ((math.dist(a[c], d[n]) + 2*(e[c]**2 + f[n]**2))**(0.5))
        temp1 += temp2
        temp4 = math.exp(-1* (math.dist(a[c], d[n]) + 2*(e[c]**2 + f[n]**2))**(0.5)) 
        temp3 += temp4  
    
    for r in range(R[n]):
        for k in range(K):
            f_term = g[i,r,k]*x*((math.dist(a[k], d[n]) + 2*(e[k]**2 + f[n]**2))**(-0.5))
            s_term = temp1/temp3
            second_term += f_term - s_term
            
    third_term = 0
    for j in range(N):
        if j != n:
            temp1 = x*((math.dist(d[i], d[j])+2*(x**2+f[j]**2))**(-0.5))
            temp2 = y[i,j]*w + math.exp(w*(1-((math.dist(d[i], d[j])+2*(x**2+f[j]**2))**(0.5))))/ (1+math.exp(w*(1-((math.dist(d[i], d[j])+2*(x**2+f[j]**2))**(0.5)))))
            
            third_term += temp1*temp2
    return first_term -2*second_term - 2*third_term

In [186]:
def compute_tomega(y, d, f, N):
    sol = optimize.root(lambda x: tomega_func(x, y, d, f, N), x0 = [0])    
    return sol.x

In [184]:
def tomega_func(x, y, d, f, N):
    # x is \tilde(omega) of interest
    # y is connection matrix NxN
    # d is the vector of \tilde(nu_1n)
    # f is the vector of \tilde(nu_2n)
    first_term = 1/x
    second_term = 0
    for i in range(N):
        for j in range(i+1, N):
            f_term = y[i,j] * (1 - (math.dist(d[i], d[j])+2*(f[i]**2+f[j]**2))**(-0.5))
            s_term = (1 - (math.dist(d[i], d[j])+2*(f[i]**2+f[j]**2))**(0.5))
            t_term = 1+ math.exp(x*(1 - (math.dist(d[i], d[j])+2*(f[i]**2+f[j]**2))**(-0.5)))
            second_term += f_term - s_term/t_term
    return first_term + second_term

In [ ]:
def compute_tphikv(k,v, p, g,x, N, R):
    #k,v are dimensions
    # p is phi
    # g is the matrix of \tilde(tau_nrk) NxRxK
    # x is the matrix of word ???
    sum_ = 0
    for n in range(N):
        for r in range(R[n]):
            toadd = g[n,r, k]*x[i,r, v]
            sum_ += toadd
    return sum_ + p

In [8]:
def compute_ttauirk(i, r, k, a, d, e, f, x, t, K, V):
    # i,r,k dimensions
    # a is the vector of \tilde(eta_1k)
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # x is the matrix of word
    # t is the matrix of \tilde(phi), KxV
    exp_1 = -1 -1*((math.dist(a[k],d[i])+2*(e[k]**2+f[i]**2))**0.5)
    exp_2 = 0
    exp_2_sum = 0
    for j in range(V):
        exp_2_sum += t[k,j]

    for v in range(V):
        to_add = x[i,r,v]*(psi(t[k,v]) - psi(exp_2_sum))
        exp_2 += to_add
    first_term = math.exp(exp_1 + exp_2)
    second_term = 0
    for c in range(K):
        to_add = math.exp(-1*((math.dist(a[c],d[i])+2*(e[c]**2+f[i]**2))**0.5))
        second_term += to_add
    return first_term - second_term

In [12]:
def ElogPzt(a,b,c,e, K):
    # a is the vector of \tilde(eta_1k)
    # b is eta_1
    # c is eta_2
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # h is nu_1
    # m is nu_2
    sum_ = 0
    temp1 = -K*math.log(2*math.pi)
    temp2 = -2*K*math.log(c)
    
    for k in range(K):
        toadd =  e[k]**2 + np.inner(a[k]-b, a[k]-b) #????
        sum_ += toadd
    
    return temp1+temp2-sum_/(2*(c**2))

In [11]:
def ElogPz(d,f,h,m, N):
    # a is the vector of \tilde(eta_1k)
    # b is eta_1
    # c is eta_2
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # h is nu_1
    # m is nu_2
    
    sum_ = 0
    temp1 = -N*math.log(2*math.pi)
    temp2 = -2*N*math.log(m)
    
    for i in range(N):
        toadd = f[i]**2 + np.inner(d[i]-h, d[i]-h)
        sum_ += toadd
    return temp1+temp2-sum_/(2*(m**2))

In [10]:
def ElogPb(phi, x, K,V):
    # phi is phi
    # x is the matrix of \tilde(phi_kj), KxV
    sum_ = 0
    temp1 = K*math.log(math.gamma(V*phi))
    temp2 = -math.log(math.gamma(phi))*V*K
    sum_ = 0
    for k in range(K):
        sum__ = 0
        for j in range(V):
            sum__ += x[k,j]
        for v in range(V):
            sum_ += psi(x[k,v]) - psi(sum__)
            
    return sum_*(phi-1) +temp1 + temp2

In [9]:
def ElogPa(w):
# w is omega
    return -2*math.log(w)

In [20]:
def ElogPu(a, d, e, f, g, N, R, K):
    # a is the vector of \tilde(eta_1k)
    # b is eta_1
    # c is eta_2
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # g is the matrix of \tilde(tau_nrk) NxRxK
    # h is nu_1
    # m is nu_2
    sum_ = 0
    for i in range(N):
        for r in range(R[i]):
            for k in range(K):
                sum_ += -g[i,r,k]*((math.dist(a[k],d[i]), 2*(e[k]**2 +f[i]**2))**(0.5))
                
                temp1 = 0
                for c in range(K):
                    temp1 += math.exp(-((math.dist(a[c],d[i]), 2*(e[c]**2 +f[i]**2))**(0.5)))
                sum_ += -g[i,r,k]*math.log(temp1)
    return sum_
        

In [21]:
def ElogPx(g, x, phi, N, R, K, V):
    sum_ = 0
    for i in range(N):
        for r in range(R[i]):
            for k in range(K):
                temp1 = 0
                for c in range(V):
                    temp1 += phi(k,c)
                for v in range(V):
                    sum_ += g[i,r,k]*x[i, v, r]*(psi(phi[k,v]) - psi(temp1))
    return sum_
                    
                

In [22]:
def ElogPY( d, f, w,y, N):
    sum_ = 0
    for i in range(N):
        for j in range(i+1, N):
            temp1 = y[i, j] *w * (1-(math.dist(d[i], d[j]) + 2*(f[i]**2-f[j]**2))**0.5)
            temp2 = math.log(1+math.exp(-w*(1-(math.dist(d[i], d[j]) + 2*(f[i]**2-f[j]**2))**0.5)))
            sum_ += temp1 - temp2
    return sum_

In [19]:
def Elogqzt(e, K):
    temp1 = -K*math.log(2*math.pi)
    sum_ = 0
    for k in range(K):
        sum_ += math.log(e[k])
    return temp1- 2*sum_ -1

In [17]:
def Elogqz(f, N):
    temp1 = -N*math.log(2*math.pi)
    sum_ =0
    for i in range(N):
        sum_ += log(f[i])
    return temp1 - 2*math.log(sum_) -1

In [14]:
def Elogqb(phi, V, K):
    temp1 = 0 
    for k in range(K):
        temp2 = 0
        temp3 = 0
        for v in range(V):
            temp2 += phi[k,v]
        temp1 += math.log(math.gamma(temp2))
        for v in range(V):
            temp3 += (phi[k,v]-1)*(psi(phi[k,v])-psi(temp2)) - math.log(math.gamma(phi[k,v]))
        temp1 += temp3
    return temp1
        

In [15]:
def Elogqa(w):
    # here is \tilde omega
    return -math.log(2*w)

In [16]:
def Elogqu(g, N,R, K):
    sum_ = 0
    for i in range(N):
        for r in range(R[i]):
            for k in range(K):
                sum+= g[i,r,k]* log(g[i,r,k])
    return sum_

In [23]:
def KL_score(a0,b0,c0,d0,e0,f0,g0,t0,w0,omega0,h0,m0,R0,N0,V0,K0,phi0, X, Y):
    temp1 =  ElogPzt(a0,b0,c0,e0,K0) + ElogPz(d0,f0,h0,m0, N0)+ElogPb(phi0, X, K,V)+ElogPa(omega0)
    temp2 = ElogPu(a0, d0, e0, f0, g0, N0, R0, K0)+ElogPx(g0, X, phi0, N0, R0, K0, V0)+ElogPY( d0, f0, w0,Y, N0)
    temp3 = Elogqzt(e0,K0) + Elogqz(f0, N0) +Elogqb(phi0, V0, K0) + Elogqa(w0) +  Elogqu(g0, N0,R0, K0)
    return temp1 + temp2 - temp3

In [ ]:
    # a is the vector of \tilde(eta_1k)
    # b is eta_1
    # c is eta_2
    # d is the vector of \tilde(nu_1n)
    # e is the vector of \tilde(eta_2k)
    # f is the vector of \tilde(nu_2n)
    # g is the matrix of \tilde(tau_nrk) NxRxK
    # t is the matrix of \tilde(phi), KxV
    # w is \tilde(omega)
    # N = 10
    # R - number of words in character i’s document
    # h is nu_1
    # m is nu_2
    # 
    # phi is phi

In [20]:
new_a = np.zeros([3,2])
new_b = np.zeros([1,2])
new_c = np.zeros([1])
new_d = np.zeros([10,2])
new_e = np.zeros([3,1])
new_f = np.zeros([10,1])
new_g = np.zeros([10,102,])

new_w = np.zeros([1])
new_omega = np.zeros([1])
new_t = np.zeros([3, 60])
new_phi = np.zeros([1])

new_h = np.zeros([1,2])
new_m = np.zeros([1])

old_a = new_a
old_b = new_b
old_c = new_c
old_d = new_d
old_e = new_e
old_f = new_f
old_g = new_g
old_w = new_w
old_omega = new_omega
old_t = new_t
old_phi = new_phi
old_h = new_h
old_m = new_m
y
x
N
R = [97,100,98,103,89,102,89,93,94,86]
K = 3
V = 60
N = 10



    

NameError: name 'y' is not defined

In [27]:
while True:
    for k in range(K):
        new_a[k] = compute_teta1k(a0 = old_a, b0=old_b, c0=old_c, d0=old_d, e0=old_e, f0=old_f, g0=old_g, N0=N, R0=R, K0=K, k0=k)
        new_e[k] = compute_teta2k(k0=k, a0 = old_a, c0=old_c, d0=old_d,  e0=old_e,f0=old_f, g0=old_g, N0=N, R0=R, K0=K)
        for v in range(V):
            new_t[k,v] = compute_tphikv(k=k,v=v, p=old_phi, g=old_g,x=X, N=N, R=R)
    
    for i in range(N):
        new_d[i] = compute_tnu1n(n0=i, a0=old_a, d0=old_d,e0=old_e, f0=old_f, g0=old_g, h0=old_h, w0 = old_w, m0 = old_m, y0=Y,N0=N, R0=R, K0=K)
        new_f[i] = compute_tnu2n(n=i, m=old_m, a=old_a,d=old_d, e=old_e,f=old_f,g=old_g, w=old_w, K=K, R=R, N=N)
        for r in range(R[n]):
            for k in range(K):
                new_g = compute_ttauirk(i, r, k, a=old_a, d=old_d, e=old_e, f=old_f, x=X, t=old_t, K=K, V=V)
    new_w = compute_tomega(y=Y, d=old_d, f=old_f, N=N)
    new_b = compute_eta1(old_a)
    new_c = compute_eta2(old_a, old_b, old_e, K)
    new_omega = compute_omega()
    new_phi = compute_phi(old_t, V, K)
    new_h = compute_nu1(old_d)
    new_m =compute_nu2(old_d, old_f, old_h)
    
    KL_s = KL_score(a0=new_a, b0=new_b, c0=new_c,d0=new_d,e0=new_e,f0 = new_f,g0 = new_g, w0= new_w, 
                    omega0= new_omega, t0= new_t, phi0 = new_phi, h0 = new_h, m0= new_m, Y=Y, X=X, R0=R,N0=N,K0=K,V0=V)
    print(KL_s)
    old_a = new_a
    old_b = new_b
    old_c = new_c
    old_d = new_d
    old_e = new_e
    old_f = new_f
    old_g = new_g
    old_w = new_w
    old_omega = new_omega
    old_t = new_t
    old_phi = new_phi
    old_h = new_h
    old_m = new_m
    
        

NameError: name 'compute_teta1k' is not defined

In [26]:
K = 3

In [17]:
a = np.zeros(1)

In [33]:
NaN*2

NameError: name 'NaN' is not defined

0.0